Imports and Setup

In [1]:
import pandas as pd
import numpy as np
import time
from collections import Counter

# Preprocessing
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.base import BaseEstimator, ClassifierMixin

# Set a consistent random state for reproducibility
RANDOM_STATE = 42
torch.manual_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


Part I: Data Preprocessing

In [2]:
print("--- Starting Part 1: Data Preprocessing ---")

# --- Step 2.1: Loading and Labeling [cite: 47] ---
# Define the 41 feature names plus the 'attack' and 'level' columns
column_names = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes',
    'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in',
    'num_compromised', 'root_shell', 'su_attempted', 'num_root',
    'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds',
    'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate',
    'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
    'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
    'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate', 'dst_host_srv_serror_rate',
    'dst_host_rerror_rate', 'dst_host_srv_rerror_rate',
    'attack', 'level'
]

# Load datasets
try:
    # Update these paths to your file locations
    train_df = pd.read_csv('KDDTrain+.txt', header=None, names=column_names)
    test_df = pd.read_csv('KDDTest+.txt', header=None, names=column_names)
    print("Files loaded successfully.")
except FileNotFoundError:
    print("Error: Dataset files not found. Please check paths.")
    # In a real script, you might exit here.

# --- Step 2.2: Binary Target Mapping [cite: 49-51] ---
# Map 'normal' to 0 and all attack types to 1
train_df['attack_flag'] = train_df['attack'].apply(lambda x: 0 if x == 'normal' else 1)
test_df['attack_flag'] = test_df['attack'].apply(lambda x: 0 if x == 'normal' else 1)

# Drop original text columns
train_df = train_df.drop(columns=['attack', 'level'])
test_df = test_df.drop(columns=['attack', 'level'])

# --- Step 2.3: Categorical Feature Encoding [cite: 52-54] ---
categorical_features = ['protocol_type', 'service', 'flag']
combined_df = pd.concat([train_df, test_df], keys=['train', 'test'])

# One-hot encode
combined_encoded_df = pd.get_dummies(combined_df, columns=categorical_features)

# Separate back into train and test
train_df_encoded = combined_encoded_df.loc['train']
test_df_encoded = combined_encoded_df.loc['test']

# Align columns - ensures train and test have the exact same columns after encoding
train_cols = train_df_encoded.columns.drop('attack_flag')
test_cols = test_df_encoded.columns.drop('attack_flag')

missing_in_test = set(train_cols) - set(test_cols)
for c in missing_in_test:
    test_df_encoded[c] = 0

missing_in_train = set(test_cols) - set(train_cols)
for c in missing_in_train:
    train_df_encoded[c] = 0

test_df_encoded = test_df_encoded[train_df_encoded.columns]
test_df_encoded = test_df_encoded.reindex(columns=train_df_encoded.columns, fill_value=0)

# Separate features (X) and target (y)
X_train = train_df_encoded.drop(columns='attack_flag')
y_train = train_df_encoded['attack_flag']
X_test = test_df_encoded.drop(columns='attack_flag')
y_test = test_df_encoded['attack_flag']

# Ensure column order is identical
X_test = X_test[X_train.columns]

print(f"Data shapes before scaling: X_train: {X_train.shape}, X_test: {X_test.shape}")

# --- Step 2.4: Feature Space Normalization [cite: 55-59] ---
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame to keep column names
X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns)

print("Features normalized with Min-Max scaling.")

# --- Step 2.5: Class Imbalance Mitigation (SMOTE) [cite: 60-64] ---
print(f"Class distribution before SMOTE: {Counter(y_train)}")
smote = SMOTE(random_state=RANDOM_STATE)
# Apply SMOTE ONLY to training data
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
print(f"Class distribution after SMOTE: {Counter(y_train_smote)}")

print("--- Data Preprocessing Complete ---")

--- Starting Part 1: Data Preprocessing ---
Files loaded successfully.
Data shapes before scaling: X_train: (125973, 122), X_test: (22544, 122)
Features normalized with Min-Max scaling.
Class distribution before SMOTE: Counter({0: 67343, 1: 58630})
Class distribution after SMOTE: Counter({0: 67343, 1: 67343})
--- Data Preprocessing Complete ---


Part II: Model & Attack Definitions

In [3]:
print("--- Starting Part 2: Model & Attack Definitions ---")

# --- Model Architecture: PyTorch Logistic Regression ---
# This architecture is used for all robust models [cite: 88, 153]
class PyTorchLogisticRegression(nn.Module):
    def __init__(self, input_dim):
        super(PyTorchLogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        # Returns raw logits, not probabilities
        return self.linear(x)

# Get input dimension from our data
input_dim = X_train_smote.shape[1]
print(f"Model input dimensions set to: {input_dim}")

# --- Attack Function 1: FGSM [cite: 74-80] ---
def fgsm_attack(model, loss_function, x, y, epsilon):
    """Generates adversarial examples using FGSM."""
    x.requires_grad = True

    outputs = model(x)
    loss = loss_function(outputs, y)

    model.zero_grad()
    loss.backward()

    data_grad = x.grad.data
    sign_data_grad = data_grad.sign()
    x_adv = x + epsilon * sign_data_grad

    # Clip to valid [0, 1] range [cite: 116]
    x_adv = torch.clamp(x_adv, 0, 1)

    return x_adv.detach()

print("Defined: PyTorchLogisticRegression class, fgsm_attack function")

# --- Attack Function 2: PGD [cite: 160-186] ---
def pgd_attack(model, loss_function, x, y, epsilon, alpha, num_iter):
    """Generates adversarial examples using PGD."""
    # Start with a random perturbation
    x_adv = x.clone().detach() + torch.empty_like(x).uniform_(-epsilon, epsilon)
    x_adv = torch.clamp(x_adv, 0, 1) # Ensure valid start

    x_clean = x.clone().detach()

    for _ in range(num_iter):
        x_adv.requires_grad = True

        outputs = model(x_adv)
        loss = loss_function(outputs, y)

        model.zero_grad()
        loss.backward()

        # Take a small step
        x_adv = x_adv.detach() + alpha * x_adv.grad.sign()

        # Project perturbation back into epsilon-ball
        perturbation = torch.clamp(x_adv - x_clean, min=-epsilon, max=epsilon)
        x_adv = x_clean + perturbation

        # Clip to valid [0, 1] range
        x_adv = torch.clamp(x_adv, 0, 1)

    return x_adv.detach()

print("Defined: pgd_attack function")

# --- Helper for Weighted Averaging ---
def get_pytorch_probs(model, X_tensor):
    """Helper function to get [prob_0, prob_1] from a PyTorch model."""
    model.eval()
    X_tensor = X_tensor.to(device)
    with torch.no_grad():
        logits = model(X_tensor)
        probs_attack = torch.sigmoid(logits) # Prob of class 1
        probs_normal = 1 - probs_attack      # Prob of class 0
        return torch.cat((probs_normal, probs_attack), dim=1).cpu().numpy()

print("Defined: get_pytorch_probs helper")
print("--- Model & Attack Definitions Complete ---")

--- Starting Part 2: Model & Attack Definitions ---
Model input dimensions set to: 122
Defined: PyTorchLogisticRegression class, fgsm_attack function
Defined: pgd_attack function
Defined: get_pytorch_probs helper
--- Model & Attack Definitions Complete ---


Part III: Train Specialist Models

In [4]:
print("--- Starting Part 3: Training Specialist Models ---")

# --- Common Hyperparameters ---
NUM_EPOCHS = 10
BATCH_SIZE = 128
LEARNING_RATE = 0.001
ALPHA_LOSS = 0.5 # Balances clean vs. adversarial loss [cite: 95-97]

# FGSM Attack Hyperparameters
EPSILON_FGSM = 0.05

# PGD Attack Hyperparameters
EPSILON_PGD = 0.05
ALPHA_PGD = 0.01
NUM_ITER_PGD = 7

# --- Convert data to PyTorch Tensors and create DataLoaders ---
# Training Data (SMOTE)
X_train_tensor = torch.tensor(X_train_smote.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_smote.values, dtype=torch.float32).view(-1, 1)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Test Data (Clean, original)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)
# Note: y_test_numpy is used for scikit-learn metrics later
y_test_numpy = y_test.values

print("PyTorch DataLoaders created.")

# --- 4.1: Standard Model (Accuracy Specialist) [cite: 6-40] ---
print("\nTraining 1/3: Standard Model (Scikit-learn)...")
standard_model = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)
standard_model.fit(X_train_smote, y_train_smote)
print("Standard Model trained.")

# --- 4.2: FGSM-Robust Model (Baseline Robustness) [cite: 66-124] ---
print("\nTraining 2/3: FGSM-Robust Model (PyTorch)...")
fgsm_robust_model = PyTorchLogisticRegression(input_dim).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(fgsm_robust_model.parameters(), lr=LEARNING_RATE)

fgsm_robust_model.train()
for epoch in range(NUM_EPOCHS):
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        # 1. Attacker's Move: Generate FGSM examples
        batch_x_adv = fgsm_attack(fgsm_robust_model, criterion, batch_x, batch_y, EPSILON_FGSM)

        # 2. Defender's Countermove: Train on combined loss
        outputs_clean = fgsm_robust_model(batch_x)
        outputs_adv = fgsm_robust_model(batch_x_adv)

        loss_clean = criterion(outputs_clean, batch_y)
        loss_adv = criterion(outputs_adv, batch_y)
        loss = (ALPHA_LOSS * loss_clean) + ((1 - ALPHA_LOSS) * loss_adv)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"FGSM Model - Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {loss.item():.4f}")
print("FGSM-Robust Model trained.")

# --- 4.3: PGD-Robust Model (Robustness Specialist) [cite: 127-189] ---
print("\nTraining 3/3: PGD-Robust Model (PyTorch)...")
pgd_robust_model = PyTorchLogisticRegression(input_dim).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(pgd_robust_model.parameters(), lr=LEARNING_RATE)

pgd_robust_model.train()
for epoch in range(NUM_EPOCHS):
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        # 1. Attacker's Move: Generate PGD examples
        batch_x_adv = pgd_attack(
            pgd_robust_model, criterion, batch_x, batch_y,
            EPSILON_PGD, ALPHA_PGD, NUM_ITER_PGD
        )

        # 2. Defender's Countermove: Train on combined loss
        outputs_clean = pgd_robust_model(batch_x)
        outputs_adv = pgd_robust_model(batch_x_adv)

        loss_clean = criterion(outputs_clean, batch_y)
        loss_adv = criterion(outputs_adv, batch_y)
        loss = (ALPHA_LOSS * loss_clean) + ((1 - ALPHA_LOSS) * loss_adv)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"PGD Model - Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {loss.item():.4f}")
print("PGD-Robust Model trained.")
print("--- All Specialist Models Trained ---")

--- Starting Part 3: Training Specialist Models ---
PyTorch DataLoaders created.

Training 1/3: Standard Model (Scikit-learn)...
Standard Model trained.

Training 2/3: FGSM-Robust Model (PyTorch)...
FGSM Model - Epoch [1/10], Loss: 0.1948
FGSM Model - Epoch [2/10], Loss: 0.2590
FGSM Model - Epoch [3/10], Loss: 0.0605
FGSM Model - Epoch [4/10], Loss: 0.0722
FGSM Model - Epoch [5/10], Loss: 0.1065
FGSM Model - Epoch [6/10], Loss: 0.1392
FGSM Model - Epoch [7/10], Loss: 0.2069
FGSM Model - Epoch [8/10], Loss: 0.1376
FGSM Model - Epoch [9/10], Loss: 0.0177
FGSM Model - Epoch [10/10], Loss: 0.2056
FGSM-Robust Model trained.

Training 3/3: PGD-Robust Model (PyTorch)...
PGD Model - Epoch [1/10], Loss: 0.2982
PGD Model - Epoch [2/10], Loss: 0.1362
PGD Model - Epoch [3/10], Loss: 0.1081
PGD Model - Epoch [4/10], Loss: 0.0896
PGD Model - Epoch [5/10], Loss: 0.0465
PGD Model - Epoch [6/10], Loss: 0.0611
PGD Model - Epoch [7/10], Loss: 0.2189
PGD Model - Epoch [8/10], Loss: 0.1127
PGD Model - Epoc

Part IV: Evaluation Framework

In [5]:
print("--- Starting Part 4: Evaluation Framework ---")

# --- 5.1: Create Attack Gradient Source Model ---
# This is a 'naive' model trained only on clean data
# Its gradients are used to create the adversarial test sets [cite: 264, 271]
print("Training naive PyTorch model (for attack generation)...")
naive_pytorch_model = PyTorchLogisticRegression(input_dim).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(naive_pytorch_model.parameters(), lr=LEARNING_RATE)

naive_pytorch_model.train()
for epoch in range(NUM_EPOCHS):
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        outputs = naive_pytorch_model(batch_x)
        loss = criterion(outputs, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
print("Naive gradient source model trained.")

# --- 5.2: Generate Adversarial Testbeds  ---
print("Generating adversarial testbeds...")
naive_pytorch_model.eval()

# 1. Clean Test Set (already have X_test_tensor, X_test)
X_test_clean_tensor = X_test_tensor.to(device)
X_test_clean_numpy = X_test.values
print("1. Clean Test Set loaded.")

# 2. FGSM Adversarial Test Set
X_test_fgsm_tensor = fgsm_attack(
    naive_pytorch_model, criterion, X_test_clean_tensor, y_test_tensor.to(device), EPSILON_FGSM
).to(device)
X_test_fgsm_numpy = X_test_fgsm_tensor.cpu().detach().numpy()
print("2. FGSM Adversarial Test Set generated.")

# 3. PGD Adversarial Test Set
X_test_pgd_tensor = pgd_attack(
    naive_pytorch_model, criterion, X_test_clean_tensor, y_test_tensor.to(device),
    EPSILON_PGD, ALPHA_PGD, NUM_ITER_PGD
).to(device)
X_test_pgd_numpy = X_test_pgd_tensor.cpu().detach().numpy()
print("3. PGD Adversarial Test Set generated.")
print("All testbeds are ready.")


# --- 5.3: Define Evaluation Helper Functions ---
def evaluate_sklearn_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred, zero_division=0),
        "Recall": recall_score(y_test, y_pred, zero_division=0),
        "F1-Score": f1_score(y_test, y_pred, zero_division=0)
    }

def evaluate_pytorch_model(model, X_tensor, y_test):
    model.eval()
    with torch.no_grad():
        y_logits = model(X_tensor)
        y_probs = torch.sigmoid(y_logits)
        y_pred = (y_probs > 0.5).float().cpu().numpy()

    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred, zero_division=0),
        "Recall": recall_score(y_test, y_pred, zero_division=0),
        "F1-Score": f1_score(y_test, y_pred, zero_division=0)
    }

print("Evaluation helper functions defined.")

--- Starting Part 4: Evaluation Framework ---
Training naive PyTorch model (for attack generation)...
Naive gradient source model trained.
Generating adversarial testbeds...
1. Clean Test Set loaded.
2. FGSM Adversarial Test Set generated.
3. PGD Adversarial Test Set generated.
All testbeds are ready.
Evaluation helper functions defined.


Final Analysis & Results


In [6]:
print("--- Starting Part 5: Final Analysis ---")
print("Executing 4x3 Evaluation Matrix...")

all_results = []
testbeds_sklearn = {
    "Clean": X_test_clean_numpy,
    "FGSM Adv.": X_test_fgsm_numpy,
    "PGD Adv.": X_test_pgd_numpy
}
testbeds_pytorch = {
    "Clean": X_test_clean_tensor,
    "FGSM Adv.": X_test_fgsm_tensor,
    "PGD Adv.": X_test_pgd_tensor
}

# --- 1. Evaluate Standard Model ---
print("Evaluating 1/4: Standard Model")
for name, X in testbeds_sklearn.items():
    metrics = evaluate_sklearn_model(standard_model, X, y_test_numpy)
    all_results.append({"Model": "Standard", "Test Set": name, **metrics})

# --- 2. Evaluate FGSM-Robust Model ---
print("Evaluating 2/4: FGSM-Robust Model")
for name, X in testbeds_pytorch.items():
    metrics = evaluate_pytorch_model(fgsm_robust_model, X, y_test_numpy)
    all_results.append({"Model": "FGSM Adv.", "Test Set": name, **metrics})

# --- 3. Evaluate PGD-Robust Model ---
print("Evaluating 3/4: PGD-Robust Model")
for name, X in testbeds_pytorch.items():
    metrics = evaluate_pytorch_model(pgd_robust_model, X, y_test_numpy)
    all_results.append({"Model": "PGD Adv.", "Test Set": name, **metrics})

# --- 4. Evaluate Hybrid Model (Defensive Weighted Average) ---
print("Evaluating 4/4: Hybrid (Defensive Avg.) Model")
weights = {'std': 0.2, 'fgsm': 0.3, 'pgd': 0.5}

for name in testbeds_sklearn.keys():
    # Get probs from all 3 models for this testbed
    probs_std = standard_model.predict_proba(testbeds_sklearn[name])
    probs_fgsm = get_pytorch_probs(fgsm_robust_model, testbeds_pytorch[name])
    probs_pgd = get_pytorch_probs(pgd_robust_model, testbeds_pytorch[name])

    # Apply the weighted average
    final_probs = (
        (weights['std'] * probs_std) +
        (weights['fgsm'] * probs_fgsm) +
        (weights['pgd'] * probs_pgd)
    )
    final_preds = np.argmax(final_probs, axis=1)

    # Calculate metrics
    metrics = {
        "Accuracy": accuracy_score(y_test_numpy, final_preds),
        "Precision": precision_score(y_test_numpy, final_preds, zero_division=0),
        "Recall": recall_score(y_test_numpy, final_preds, zero_division=0),
        "F1-Score": f1_score(y_test_numpy, final_preds, zero_division=0)
    }
    all_results.append({"Model": "Hybrid (Avg.)", "Test Set": name, **metrics})

print("Evaluation complete.")

# --- Present the Final Table ---
results_df = pd.DataFrame(all_results)
final_table = results_df.set_index(["Model", "Test Set"])

pd.set_option('display.float_format', '{:.4f}'.format)
print("\n\n--- FINAL COMPREHENSIVE RESULTS [cite: 293-298] ---")
print(final_table.to_markdown(floatfmt=".4f"))

--- Starting Part 5: Final Analysis ---
Executing 4x3 Evaluation Matrix...
Evaluating 1/4: Standard Model
Evaluating 2/4: FGSM-Robust Model
Evaluating 3/4: PGD-Robust Model
Evaluating 4/4: Hybrid (Defensive Avg.) Model


c:\Users\manid\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\manid\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\manid\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\manid\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\manid\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\valid

Evaluation complete.


--- FINAL COMPREHENSIVE RESULTS [cite: 293-298] ---
|                                |   Accuracy |   Precision |   Recall |   F1-Score |
|:-------------------------------|-----------:|------------:|---------:|-----------:|
| ('Standard', 'Clean')          |     0.7547 |      0.9169 |   0.6258 |     0.7439 |
| ('Standard', 'FGSM Adv.')      |     0.3201 |      0.4122 |   0.4562 |     0.4331 |
| ('Standard', 'PGD Adv.')       |     0.3267 |      0.4170 |   0.4594 |     0.4372 |
| ('FGSM Adv.', 'Clean')         |     0.7382 |      0.9148 |   0.5955 |     0.7214 |
| ('FGSM Adv.', 'FGSM Adv.')     |     0.7045 |      0.8943 |   0.5453 |     0.6775 |
| ('FGSM Adv.', 'PGD Adv.')      |     0.7055 |      0.8948 |   0.5469 |     0.6789 |
| ('PGD Adv.', 'Clean')          |     0.7390 |      0.9147 |   0.5972 |     0.7226 |
| ('PGD Adv.', 'FGSM Adv.')      |     0.7066 |      0.8940 |   0.5497 |     0.6808 |
| ('PGD Adv.', 'PGD Adv.')       |     0.7081 |      0.8949 |   0

In [7]:


def analyze_game_matrix(payoff_matrix_df):
    """
    Analyzes a payoff matrix (as a DataFrame) to find the maximin strategy
    and check for a pure strategy (saddle point).
    
    Args:
    payoff_matrix_df (pd.DataFrame): DataFrame where rows are defender strategies
                                     (models) and columns are attacker strategies
                                     (test sets). Values are defender's payoff
                                     (accuracy).
    """
    
    payoff_matrix = payoff_matrix_df.values
    model_names = payoff_matrix_df.index
    attack_names = payoff_matrix_df.columns

    print("--- 4x3 Payoff Matrix (Defender's Accuracy) ---")
    print(payoff_matrix_df.to_markdown(floatfmt=".4f"))
    print("\n")
    
    # 1. Find Defender's worst-case for each strategy (row minimums)
    row_mins = np.min(payoff_matrix, axis=1)
    
    # 2. Find Defender's best "worst-case" (maximin)
    maximin = np.max(row_mins)
    maximin_model_index = np.argmax(row_mins)
    maximin_model_name = model_names[maximin_model_index]
    
    print("--- Defender's (Our) Analysis (Maximin) ---")
    for i, model in enumerate(model_names):
        print(f"Model: {model:<15} | Worst-Case Accuracy (Row Min): {row_mins[i]:.4f}")
    
    print(f"\nBest 'Worst-Case' (Maximin): {maximin:.4f}")
    print(f"Optimal Defender Strategy (Maximin): Deploy '{maximin_model_name}'")

    # 3. Find Attacker's worst-case for each strategy (column maximums)
    # (Attacker wants to minimize our accuracy, so their "worst" case is the
    # highest accuracy they can be forced to allow)
    col_maxs = np.max(payoff_matrix, axis=0)
    
    # 4. Find Attacker's best "worst-case" (minimax)
    minimax = np.min(col_maxs)
    minimax_attack_index = np.argmin(col_maxs)
    minimax_attack_name = attack_names[minimax_attack_index]
    
    print("\n--- Attacker's Analysis (Minimax) ---")
    for i, attack in enumerate(attack_names):
        print(f"Attack: {attack:<10} | Worst-Case for Attacker (Col Max): {col_maxs[i]:.4f}")
    
    print(f"\nAttacker's Best 'Worst-Case' (Minimax): {minimax:.4f}")
    
    # 5. Check for Pure Strategy (Saddle Point)
    print("\n--- Nash Equilibrium Analysis ---")
    if maximin == minimax:
        print(f"Game Type: Pure Strategy (Saddle Point) Exists!")
        print(f"  - Defender's Best Strategy: Always play '{maximin_model_name}'")
        print(f"  - Attacker's Best Strategy: Always play '{minimax_attack_name}'")
        print(f"  - Value of the Game: {maximin:.4f}")
    else:
        print(f"Game Type: No Pure Strategy (Saddle Point).")
        print("This means a mixed strategy (using models with different probabilities) might be optimal.")
        print(f"However, the most robust (Maximin) strategy is still to deploy '{maximin_model_name}'")
        print(f"as it guarantees you a minimum accuracy of {maximin:.4f} no matter what the attacker does.")

# --- Main execution ---
# This code assumes 'final_table' (a DataFrame) was created in the cell above.
try:
    # 'final_table' has a MultiIndex (Model, Test Set).
    # We unstack it to create the 4x3 payoff matrix, using Accuracy.
    payoff_df = final_table['Accuracy'].unstack()
    
    # Ensure the columns are in the correct logical order
    payoff_df = payoff_df[['Clean', 'FGSM Adv.', 'PGD Adv.']]
    
    analyze_game_matrix(payoff_df)

except NameError as e:
    print(f"Error: Could not find the 'final_table' DataFrame.")
    print(f"Details: {e}")
    print("Please make sure you have run the previous cell (cell 11) successfully.")
except KeyError:
    print("Error: The 'final_table' DataFrame is missing 'Accuracy' or the expected Test Set names.")
    print("Please check the results from the previous cell.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

--- 4x3 Payoff Matrix (Defender's Accuracy) ---
| Model         |   Clean |   FGSM Adv. |   PGD Adv. |
|:--------------|--------:|------------:|-----------:|
| FGSM Adv.     |  0.7382 |      0.7045 |     0.7055 |
| Hybrid (Avg.) |  0.7385 |      0.6891 |     0.6898 |
| PGD Adv.      |  0.7390 |      0.7066 |     0.7081 |
| Standard      |  0.7547 |      0.3201 |     0.3267 |


--- Defender's (Our) Analysis (Maximin) ---
Model: FGSM Adv.       | Worst-Case Accuracy (Row Min): 0.7045
Model: Hybrid (Avg.)   | Worst-Case Accuracy (Row Min): 0.6891
Model: PGD Adv.        | Worst-Case Accuracy (Row Min): 0.7066
Model: Standard        | Worst-Case Accuracy (Row Min): 0.3201

Best 'Worst-Case' (Maximin): 0.7066
Optimal Defender Strategy (Maximin): Deploy 'PGD Adv.'

--- Attacker's Analysis (Minimax) ---
Attack: Clean      | Worst-Case for Attacker (Col Max): 0.7547
Attack: FGSM Adv.  | Worst-Case for Attacker (Col Max): 0.7066
Attack: PGD Adv.   | Worst-Case for Attacker (Col Max): 0.7081

Att